In [38]:
import requests
import pandas as pd
import numpy as np
import json
import geopandas as gpd
import ast

# 1. Load data 

In [9]:
restaurants_df = gpd.read_file("dataset/processed_datasets/restaurants.geojson")
nightlife_df = gpd.read_file("dataset/processed_datasets/nightlife_locations.geojson")

# 2. Get crowd density data

In [10]:
#Function to get crowd density data from besttime using restaurant name and address
def get_cd_json(name, address, is_midnight_restaurant):
    if is_midnight_restaurant == 'False':
        return np.nan

    url = "https://besttime.app/api/v1/forecasts"
    params = {
        'api_key_private': 'pri_213fe6a21137499da100e7bf8b3553f5',
        'venue_name': name,
        'venue_address': address,
    }

    response = requests.request("POST", url, params=params)
    return response.json()

In [11]:
# Function to get crowd density data from besttime using nightlife location name and address
def get_cd_json_(name, address):
    url = "https://besttime.app/api/v1/forecasts"
    params = {
        'api_key_private': 'pri_213fe6a21137499da100e7bf8b3553f5',
        'venue_name': name,
        'venue_address': address,
    }

    response = requests.request("POST", url, params=params)
    return response.json()

In [12]:
restaurants_df['cd_json'] = restaurants_df.apply(lambda x: get_cd_json(x['name'], x['address'], x['midnight_restaurant']), axis=1)

In [ ]:
nightlife_df['cd_json'] = nightlife_df.apply(lambda x: get_cd_json(x['name'], x['address']), axis=1)

In [40]:
# Initialize empty columns to store Friday and Saturday crowd density data
restaurants_df['fri_cd'], restaurants_df['sat_cd'] = np.nan, np.nan
nightlife_df['fri_cd'], nightlife_df['sat_cd'] = np.nan, np.nan

In [75]:
def get_fri_sat_cd(row):
    fri_cd, sat_cd = np.nan, np.nan
    cd_json = row['cd_json']
    if type(cd_json) != dict:
        return np.nan, np.nan

    if cd_json['status'] == 'OK':
        analysis = cd_json['analysis']
        for day in analysis: 
            day_text = day['day_info']['day_text']
            day_raw = day['day_raw']
            if day_text == 'Friday':
                fri_cd = day_raw
            if day_text == 'Saturday':
                sat_cd = day_raw
    return fri_cd, sat_cd

In [80]:
restaurants_df[['fri_cd', 'sat_cd']] = restaurants_df.apply(lambda x: get_fri_sat_cd(x), axis=1, result_type='expand')
nightlife_df[['fri_cd', 'sat_cd']] = nightlife_df.apply(lambda x: get_fri_sat_cd(x), axis=1, result_type='expand')

In [39]:
def get_hour_cd(row, index):
    if pd.isna(row['sat_cd']):
        return np.nan

    return ast.literal_eval(row['sat_cd'])[index]

In [40]:
starting_ind = 16
for hour in ['10pm','11pm','12pm','1am','2am']:
    restaurants_df[f"{hour}_cd"] = restaurants_df.apply(lambda x: get_hour_cd(x,starting_ind), axis=1)
    starting_ind += 1

In [42]:
restaurants_df.to_csv("dataset/besttime/processed_restaurants.csv")